In [ ]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(0)

In [ ]:
MEAN_A = [0, 0]
MEAN_B = [2, 2]
MEAN_C = [-4, 0]
COV_A = [[1, 1], [1, 3]]
COV_B = [[4, 1], [1, 2]]
COV_C = [[2, 1], [1, 2]]
N_A = 60
N_B = 30
N_C = 10

In [ ]:
X_A = np.concatenate((np.random.multivariate_normal(MEAN_A, COV_A, N_A),
                     np.zeros((N_A, 1))), 
                     axis = 1)
X_B = np.concatenate((np.concatenate((np.random.multivariate_normal(MEAN_B, COV_B, N_B),
                                      np.random.multivariate_normal(MEAN_C, COV_C, N_C))),
                     np.ones((N_B + N_C, 1))),
                     axis = 1)

In [ ]:
df = pd.DataFrame(np.concatenate((X_A, X_B)), columns = ['feature_1', 'feature_2', 'label'])

In [ ]:
df

In [ ]:
fig, ax = plt.subplots()
_ = ax.scatter(df[df.label == 0].feature_1, df[df.label == 0].feature_2, color = 'C0', alpha = 0.5)
_ = ax.scatter(df[df.label == 1].feature_1, df[df.label == 1].feature_2, color = 'C1', alpha = 0.5)

In [ ]:
df = df.sample(frac = 1)
df.index = range(len(df))
df_training = df[0:math.ceil(0.8 * len(df))].copy()
df_test = df[math.ceil(0.8 * len(df)):].copy()

In [ ]:
# TODO: when training and test, plot boundaries of the classes